In [ ]:
import pandas as pd
location_df = pd.read_csv("loc_df.csv", sep='\t')
actual_df = pd.read_csv("actual_sf.csv", sep='\t')

In [ ]:
data = actual_df.append(location_df)
data.to_csv("final_data.csv", sep='\t')

In [ ]:
import pandas as pd
data = pd.read_csv("final_data.csv", sep='\t')

In [ ]:
data_hail = data[data['Trip_type '] == 1]

In [ ]:
data_hail = data_hail[(data_hail['lpep_pickup_datetime']>'06/01/2016 00:00:00 AM') & (data_hail['lpep_pickup_datetime']<='06/30/2016 23:59:59 PM')]
len(data_hail)

In [ ]:
data_hail = data_hail.drop(columns=['Unnamed: 0','Unnamed: 0.1', 'Store_and_fwd_flag', 'RateCodeID', 'Extra', 'MTA_tax', 'Tip_amount', 'Tolls_amount', 'Ehail_fee', 'improvement_surcharge', 'Total_amount'])
# data_hail.columns.values

In [ ]:
data_hail.to_csv("newd/hailForStats.csv", sep='\t')

In [ ]:
data_hail.sample(n=1000000).describe()

In [ ]:
data_hail = data_hail[ (data_hail['Pickup_longitude'] >= -74.2748) & (data_hail['Pickup_longitude'] <= -73.70741) & \
              (data_hail['Pickup_latitude'] >= 40.45326) & (data_hail['Pickup_latitude'] <= 40.94788) & \
              (data_hail['Dropoff_longitude'] >= -74.2748) & (data_hail['Dropoff_longitude'] <= -73.70741) & \
              (data_hail['Dropoff_latitude'] >= 40.45326) & (data_hail['Dropoff_latitude'] <= 40.94788)]

In [ ]:
actual_data = data_hail[data_hail['Trip_distance']!=0.0]
out_tdist = data_hail[data_hail['Trip_distance']==0.0]
# type(out_tdist)

import mpu

def get_distance(lat1, lon1, lat2, lon2):
    dist = mpu.haversine_distance((lat1, lon1), (lat2, lon2))
    return dist

out_tdist['Trip_distance']=out_tdist.apply(lambda row:get_distance(row['Pickup_latitude'],row['Pickup_longitude'],row['Dropoff_latitude'],row['Dropoff_longitude']),axis=1)
out_tdist[["Pickup_longitude", "Pickup_latitude", "Dropoff_longitude", "Dropoff_latitude", "Trip_distance", "Fare_amount"]]

In [ ]:
data_hail = actual_data.append(out_tdist)
print(len(data_hail))

In [ ]:
import numpy as np

spl = []
spl.append(data_hail.iloc[:1000000])
spl.append(data_hail.iloc[1000000:])
len(spl)

In [ ]:
lf = spl[1]
actual_data = lf [lf['Fare_amount'] >= 0]
out_fare = lf [lf['Fare_amount'] < 0]
len(out_fare)

In [ ]:
df = pd.DataFrame(columns=list(out_fare.columns.values))

for i in range(len(out_fare)):
    test_df = out_fare.iloc[i]
    tf = data_hail[ (data_hail['Pickup_longitude'] == test_df['Pickup_longitude']) & (data_hail['Pickup_latitude'] == test_df['Pickup_latitude']) \
         & (data_hail['Dropoff_longitude']== test_df['Dropoff_longitude']) & (data_hail['Dropoff_latitude'] == test_df['Dropoff_latitude']) \
         & (data_hail['Trip_distance']== test_df['Trip_distance']) ]
    if len(tf) > 1:
        test_df['Fare_amount']= tf[ tf['Fare_amount'] > 0]['Fare_amount'].mean()
        df = df.append(test_df)

In [ ]:
data1 = actual_data.append(df)
len(data1)

In [ ]:
data_hail = data_hail.append(data1)
len(data_hail)


In [ ]:
data_hail['lpep_pickup_datetime'] = pd.to_datetime(data_hail['lpep_pickup_datetime'],errors="coerce")
data_hail['Lpep_dropoff_datetime'] = pd.to_datetime(data_hail['Lpep_dropoff_datetime'],errors="coerce")

In [ ]:
import calendar

data_hail['pickup_date']= data_hail['lpep_pickup_datetime'].dt.date
data_hail['pickup_day']=data_hail['lpep_pickup_datetime'].apply(lambda x:x.day)
data_hail['pickup_hour']=data_hail['lpep_pickup_datetime'].apply(lambda x:x.hour)
data_hail['pickup_day_of_week']=data_hail['lpep_pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
data_hail['pickup_month']=data_hail['lpep_pickup_datetime'].apply(lambda x:x.month)
data_hail['pickup_year']=data_hail['lpep_pickup_datetime'].apply(lambda x:x.year)

In [ ]:
data_hail.head()

In [ ]:
data_hail.to_csv("newd/hail_df.csv", sep='\t')

In [ ]:
data_dispatch = data[data['Trip_type '] == 2]

In [ ]:
data_dispatch = data_dispatch.drop(columns=['Unnamed: 0','Unnamed: 0.1', 'Store_and_fwd_flag', 'RateCodeID', 'Extra', 'MTA_tax', 'Tip_amount', 'Tolls_amount', 'Ehail_fee', 'improvement_surcharge', 'Total_amount'])

In [ ]:
len(data_dispatch)
data_dispatch.describe()

In [ ]:
data_dispatch = data_dispatch[ (data_dispatch['Pickup_longitude'] >= -74.2748) & (data_dispatch['Pickup_longitude'] <= -73.70741) & \
              (data_dispatch['Pickup_latitude'] >= 40.45326) & (data_dispatch['Pickup_latitude'] <= 40.94788) & \
              (data_dispatch['Dropoff_longitude'] >= -74.2748) & (data_dispatch['Dropoff_longitude'] <= -73.70741) & \
              (data_dispatch['Dropoff_latitude'] >= 40.45326) & (data_dispatch['Dropoff_latitude'] <= 40.94788)]

In [ ]:
actual_data = data_dispatch[data_dispatch['Trip_distance']!=0.0]
out_tdist = data_dispatch[data_dispatch['Trip_distance']==0.0]
# type(out_tdist)

import mpu

def get_distance(lat1, lon1, lat2, lon2):
    dist = mpu.haversine_distance((lat1, lon1), (lat2, lon2))
    return dist

out_tdist['Trip_distance']=out_tdist.apply(lambda row:get_distance(row['Pickup_latitude'],row['Pickup_longitude'],row['Dropoff_latitude'],row['Dropoff_longitude']),axis=1)
out_tdist[["Pickup_longitude", "Pickup_latitude", "Dropoff_longitude", "Dropoff_latitude", "Trip_distance", "Fare_amount"]]

In [ ]:
data_dispatch = actual_data.append(out_tdist)
print(len(data_dispatch))

In [ ]:
actual_data = data_dispatch [data_dispatch['Fare_amount'] >= 0]
out_fare = data_dispatch [data_dispatch['Fare_amount'] < 0]
len(out_fare)

In [ ]:
df = pd.DataFrame(columns=list(out_fare.columns.values))

for i in range(len(out_fare)):
    test_df = out_fare.iloc[i]
    tf = data_dispatch[ (data_dispatch['Pickup_longitude'] == test_df['Pickup_longitude']) & (data_dispatch['Pickup_latitude'] == test_df['Pickup_latitude']) \
         & (data_dispatch['Dropoff_longitude']== test_df['Dropoff_longitude']) & (data_dispatch['Dropoff_latitude'] == test_df['Dropoff_latitude']) \
         & (data_dispatch['Trip_distance']== test_df['Trip_distance']) ]
    if len(tf) > 1:
        test_df['Fare_amount']= tf[ tf['Fare_amount'] > 0]['Fare_amount'].mean()
        df = df.append(test_df)

In [ ]:
data_dispatch = actual_data.append(df)
len(data_dispatch)

In [ ]:
data_dispatch['lpep_pickup_datetime'] = pd.to_datetime(data_dispatch['lpep_pickup_datetime'],errors="coerce")
data_dispatch['Lpep_dropoff_datetime'] = pd.to_datetime(data_dispatch['Lpep_dropoff_datetime'],errors="coerce")

In [ ]:
import calendar

data_dispatch['pickup_date']= data_dispatch['lpep_pickup_datetime'].dt.date
data_dispatch['pickup_day']=data_dispatch['lpep_pickup_datetime'].apply(lambda x:x.day)
data_dispatch['pickup_hour']=data_dispatch['lpep_pickup_datetime'].apply(lambda x:x.hour)
data_dispatch['pickup_day_of_week']=data_dispatch['lpep_pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
data_dispatch['pickup_month']=data_dispatch['lpep_pickup_datetime'].apply(lambda x:x.month)
data_dispatch['pickup_year']=data_dispatch['lpep_pickup_datetime'].apply(lambda x:x.year)

In [ ]:
data_dispatch.head()

In [ ]:
data_dispatch.to_csv("newd/dispatch_df.csv", sep='\t')